# BEER thumbnails

This notebook generates the thumbnails used in the BEER user guide.

In [ ]:
import matplotlib.pyplot as plt

import scipp as sc

from ess.beer import BeerModMcStasWorkflow
from ess.beer.data import mcstas_duplex
from ess.reduce.nexus.types import Filename, SampleRun
from ess.beer.types import *


In [ ]:
wf = BeerModMcStasWorkflow()
wf[Filename[SampleRun]] = mcstas_duplex(9)
histogram = wf.compute(DetectorData[SampleRun])['bank1'].hist(two_theta=1000, event_time_offset=1000)

In [ ]:
def basic_powder_plot(style: str):
    with plt.style.context(style):
        fig, ax = plt.subplots(layout='constrained', figsize=(3, 2.5))
        _ = histogram.plot(ax=ax, norm='log')
        ax.set_xlim((0.045,0.12))
        ax.set_xlabel(r'$t$ [µs]')
        ax.set_ylabel(r'$I(t)$')
    return fig

In [ ]:
fig = basic_powder_plot('default')
fig.savefig(
    "../../docs/_static/thumbnails/beer_mcstas_light.svg",
    transparent=True,
)
fig

In [ ]:
fig = basic_powder_plot('dark_background')
fig.savefig(
    "../../docs/_static/thumbnails/beer_mcstas_dark.svg",
    transparent=True,
)
fig

In [ ]:
detector_names = ["mantle", "endcap_forward", "endcap_backward", "high_resolution"]
two_theta_bins = [
    sc.linspace(dim="two_theta", unit="rad", start=0.77, stop=2.36, num=70),
    sc.linspace(dim="two_theta", unit="rad", start=0.24, stop=0.71, num=30),
    sc.linspace(dim="two_theta", unit="rad", start=2.42, stop=2.91, num=50),
    sc.linspace(dim="two_theta", unit="rad", start=2.91, stop=3.11, num=10),
]
parameter_table = pd.DataFrame(
    {NeXusDetectorName: detector_names,
     TwoThetaBins: two_theta_bins,
     },
    index=detector_names
).rename_axis(index='detector')

all_detector_workflow = workflow.copy()
mapped = all_detector_workflow[IofDspacingTwoTheta].map(parameter_table)
all_detector_workflow[IofDspacingTwoTheta] = mapped.reduce(func=powder.grouping.collect_detectors)

result = all_detector_workflow.compute(IofDspacingTwoTheta)

In [ ]:
histogram = result.bin(dspacing=80).hist()

In [ ]:
def advanced_powder_plot(style: str):
    with plt.style.context(style):
        fig, ax = plt.subplots(layout='constrained', figsize=(3, 2.5))
        pf = pp.imagefigure(*(pp.Node(da) for da in histogram.values()), norm='log', cbar=True, ax=ax)
        pf.view.colormapper.ylabel = None
        ax.set_xlabel(r'$d$ [Å]')
        ax.set_ylabel(r'$2\theta$ [rad]')
    return fig

In [ ]:
fig = advanced_powder_plot('default')
fig.savefig(
    "../../docs/_static/thumbnails/dream_advanced_powder_reduction_light.svg",
    transparent=True,
)
fig

In [ ]:
fig = advanced_powder_plot('dark_background')
fig.savefig(
    "../../docs/_static/thumbnails/dream_advanced_powder_reduction_dark.svg",
    transparent=True,
)
fig